**<center><font size="25"> Occupations of Presidential Donors </font></center>**

**<center> By: Michael Dunphy </center>**
**<center> September 2020 </center>**

<figure>
<img src="https://pbs.twimg.com/media/EhNXxpAXYAkOUYX?format=jpg&name=large" width="800" style ="float:center;">
</figure>

## Introduction

An analysis that looks at the occupations of presidential donors for each party candidate from 1992 to 2016.

Visual can be found here: https://public.tableau.com/profile/michael.dunphy8764#!/vizhome/PresidentialDonorsfrom1992to2016/Occupation

Data Source:  https://www.fec.gov

Visual also makes use of SOC classification system: https://www.bls.gov/soc/2018/home.htm

Below is the code used to collect and modify the data to produce the Tableau visual.

## Code

The program makes use of:
* [Pandas](https://pandas.pydata.org/docs/)

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
# dataset used to classify occupations based on SOC
occupations = pd.read_csv('https://raw.githubusercontent.com/dawaldron/data-visualizations/master/actblue_data_2020/titleocc_xw_final.csv')

# drop unnecessary columns for this analysis
occupations.drop(columns = ['freq', 'source', 'onetcode', 'onettitle'], inplace = True)

# final occupations dataset used
occupations.head()

,soccode,soctitle,occupation
0,11-1011,Chief executives,CEO
1,11-1011,Chief executives,EXECUTIVE DIRECTOR
2,11-1011,Chief executives,PRESIDENT
3,11-1011,Chief executives,VP
4,11-1011,Chief executives,VICE PRESIDENT


In [3]:
# data set used to group occupations into SOC groupings
occ_agg = pd.read_csv('https://raw.githubusercontent.com/dawaldron/data-visualizations/master/actblue_data_2020/occgrpnm.csv')

# Only need the first two numbers of the SOC number to match subclasses of occupations
occ_agg['occgrpcd'] = occ_agg['occgrpcd'].str[:2]

# final occupational grouping dataset used
occ_agg.head()

,occgrpcd,occgrpnm2,N
0,11,Management,1
1,13,Business and financial,1
2,15,Computer and mathematical,1
3,17,Architecture and engineering,1
4,19,"Life, physical, social science",1


In [4]:
# creates cleaned data tables
def clean_data(data):
    # take the needed columns from the original data that we need for this analysis
    data_clean = data[['committee_id', 'report_type', 'contributor_name', 
                       'contributor_state', 'contributor_zip', 'contributor_occupation', 'contribution_receipt_date', 
                           'contribution_receipt_amount', 'contributor_aggregate_ytd']]

    # convert some columns to numerical values
    data_clean['contribution_receipt_amount'] = pd.to_numeric(data_clean['contribution_receipt_amount'])
    data_clean['contributor_aggregate_ytd'] = pd.to_numeric(data_clean['contributor_aggregate_ytd'])

    # clean zip code data to only having the 6 digits associated with each zip code
    data_clean['contributor_zip'] = data_clean['contributor_zip'].astype(str).str[:5]
    
    return data_clean

In [5]:
# creates a table showing the number and percentage of donors of that occupational group
def occupation_tables(csv, name):
    data = clean_data(pd.read_csv(csv))
    
    occ = pd.merge(data, occupations, left_on = 'contributor_occupation', right_on = 'occupation', how = 'inner')
    # only need the first 2 digits of the soc code to match
    occ['soccode_new'] = occ['soccode'].str[:2]
    occ_main = pd.merge(occ, occ_agg, left_on = 'soccode_new', right_on = 'occgrpcd', how = 'inner')
    occ_main.drop(columns = ['N', 'occgrpcd', 'soccode_new', 'occupation', 'soccode', 'soctitle'], inplace = True)

    count_title = pd.DataFrame(occ_main['occgrpnm2'].value_counts()).reset_index()
    count_title['%'] = count_title['occgrpnm2'] / count_title['occgrpnm2'].sum()
    count_title['year'] = name[1]
    count_title['candidate'] = name[0]
    
    data['candidate'] = name[0]
    data['year'] = name[1]

    return count_title, data


In [6]:
# create data tables for each of the candidates to be combined for the visual
dems = ['clinton_16', 'obama_12', 'obama_08', 'kerry_04', 'gore_00', 'clinton_96', 'clinton_92']
reps = ['trump_16', 'romney_12', 'mccain_08', 'bush_04', 'bush_00', 'dole_96', 'bush_92']

for x in dems:
    name = x.split('_')
    occ, data = occupation_tables("FEC DATA - DEM/" + name[1] + "/" + x + ".csv", name)
    data.to_csv('FEC DATA - DEM/' + name[1] + '/' + x + '_cleaned.csv')
    occ.to_csv('FEC DATA - DEM/' + name[1] + '/' + x + '_occupations.csv')
    
for x in reps:
    name = x.split('_')
    occ, data = occupation_tables("FEC DATA - REP/" + name[1] + "/" + x + ".csv", name)
    data.to_csv('FEC DATA - REP/' + name[1] + '/' + x + '_cleaned.csv')
    occ.to_csv('FEC DATA - REP/' + name[1] + '/' + x + '_occupations.csv')

Additional cleaning and grouping of the data was done in other jupyter files which can be found in each data folder.

Anaylsis by: Michael Dunphy (he/him/his)

[Twitter](https://twitter.com/mtdunphy)
[Github](https://github.com/mtdunphy-umd)
[Tableau](https://public.tableau.com/profile/michael.dunphy8764)